# 🔬 Phân tích Quá trình Xử lý Ảnh - Feature Extractor

Notebook này minh họa từng bước xử lý ảnh trong hàm `extract_eye_features()` để trích xuất 25 features từ ảnh mắt.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from utils.feature_extractor import extract_eye_features

plt.style.use('default')
plt.rcParams['figure.figsize'] = (15, 10)

## 📂 Load Sample Images

In [ ]:
# Load sample images
open_path = 'data/eyes/open'
closed_path = 'data/eyes/closed'

# Get sample images
open_files = [f for f in os.listdir(open_path) if f.endswith(('.jpg', '.png'))][:3]
closed_files = [f for f in os.listdir(closed_path) if f.endswith(('.jpg', '.png'))][:3]

# Load images
open_images = [cv2.imread(os.path.join(open_path, f), cv2.IMREAD_GRAYSCALE) for f in open_files]
closed_images = [cv2.imread(os.path.join(closed_path, f), cv2.IMREAD_GRAYSCALE) for f in closed_files]

print(f"Loaded {len(open_images)} open eye images")
print(f"Loaded {len(closed_images)} closed eye images")

## 👁️ Original Images Comparison

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# Open eyes
for i, img in enumerate(open_images):
    axes[0, i].imshow(img, cmap='gray')
    axes[0, i].set_title(f'Open Eye {i+1}')
    axes[0, i].axis('off')

# Closed eyes
for i, img in enumerate(closed_images):
    axes[1, i].imshow(img, cmap='gray')
    axes[1, i].set_title(f'Closed Eye {i+1}')
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()

## 🔧 Step 1: Image Preprocessing - Resize to 32x32

In [ ]:
# Resize images to standard 32x32
open_resized = [cv2.resize(img, (32, 32)) for img in open_images]
closed_resized = [cv2.resize(img, (32, 32)) for img in closed_images]

fig, axes = plt.subplots(2, 3, figsize=(12, 6))

for i, img in enumerate(open_resized):
    axes[0, i].imshow(img, cmap='gray')
    axes[0, i].set_title(f'Open 32x32 - {i+1}')
    axes[0, i].axis('off')

for i, img in enumerate(closed_resized):
    axes[1, i].imshow(img, cmap='gray')
    axes[1, i].set_title(f'Closed 32x32 - {i+1}')
    axes[1, i].axis('off')

plt.suptitle('Step 1: Standardized 32x32 Images')
plt.tight_layout()
plt.show()

## 📊 Step 2: Statistical Features Analysis

In [ ]:
def analyze_statistical_features(images, label):
    stats = []
    for img in images:
        stats.append({
            'mean': np.mean(img),
            'std': np.std(img),
            'var': np.var(img),
            'min': np.min(img),
            'max': np.max(img),
            'median': np.median(img)
        })
    return stats

open_stats = analyze_statistical_features(open_resized, 'Open')
closed_stats = analyze_statistical_features(closed_resized, 'Closed')

# Plot comparison
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
metrics = ['mean', 'std', 'var', 'min', 'max', 'median']

for i, metric in enumerate(metrics):
    row, col = i // 3, i % 3
    
    open_vals = [s[metric] for s in open_stats]
    closed_vals = [s[metric] for s in closed_stats]
    
    x = np.arange(len(open_vals))
    width = 0.35
    
    axes[row, col].bar(x - width/2, open_vals, width, label='Open', alpha=0.7, color='green')
    axes[row, col].bar(x + width/2, closed_vals, width, label='Closed', alpha=0.7, color='red')
    axes[row, col].set_title(f'{metric.capitalize()}')
    axes[row, col].legend()
    axes[row, col].set_xticks(x)
    axes[row, col].set_xticklabels([f'Img {i+1}' for i in range(len(open_vals))])

plt.suptitle('Step 2: Statistical Features Comparison')
plt.tight_layout()
plt.show()

print("📈 Statistical Features Summary:")
print(f"Open Eyes - Mean intensity: {np.mean([s['mean'] for s in open_stats]):.1f}")
print(f"Closed Eyes - Mean intensity: {np.mean([s['mean'] for s in closed_stats]):.1f}")
print(f"Open Eyes - Std deviation: {np.mean([s['std'] for s in open_stats]):.1f}")
print(f"Closed Eyes - Std deviation: {np.mean([s['std'] for s in closed_stats]):.1f}")

## 🎯 Step 3: Texture Features - Center Region Analysis

In [ ]:
fig, axes = plt.subplots(2, 6, figsize=(18, 6))

for i, img in enumerate(open_resized):
    # Original
    axes[0, i*2].imshow(img, cmap='gray')
    axes[0, i*2].set_title(f'Open {i+1} - Full')
    axes[0, i*2].axis('off')
    
    # Center region (12:20, 12:20)
    center = img[12:20, 12:20]
    axes[0, i*2+1].imshow(center, cmap='gray')
    axes[0, i*2+1].set_title(f'Center Region')
    axes[0, i*2+1].axis('off')
    
    # Add rectangle to show center region
    rect = plt.Rectangle((12, 12), 8, 8, linewidth=2, edgecolor='red', facecolor='none')
    axes[0, i*2].add_patch(rect)

for i, img in enumerate(closed_resized):
    # Original
    axes[1, i*2].imshow(img, cmap='gray')
    axes[1, i*2].set_title(f'Closed {i+1} - Full')
    axes[1, i*2].axis('off')
    
    # Center region
    center = img[12:20, 12:20]
    axes[1, i*2+1].imshow(center, cmap='gray')
    axes[1, i*2+1].set_title(f'Center Region')
    axes[1, i*2+1].axis('off')
    
    # Add rectangle
    rect = plt.Rectangle((12, 12), 8, 8, linewidth=2, edgecolor='red', facecolor='none')
    axes[1, i*2].add_patch(rect)

plt.suptitle('Step 3: Texture Features - Center vs Periphery Analysis')
plt.tight_layout()
plt.show()

# Calculate texture features
print("🎯 Texture Features Analysis:")
for i, (open_img, closed_img) in enumerate(zip(open_resized, closed_resized)):
    open_center = open_img[12:20, 12:20]
    closed_center = closed_img[12:20, 12:20]
    
    open_contrast = np.mean(open_center) - np.mean(open_img)
    closed_contrast = np.mean(closed_center) - np.mean(closed_img)
    
    print(f"Image {i+1} - Open contrast: {open_contrast:.2f}, Closed contrast: {closed_contrast:.2f}")

## 🔍 Step 4: Edge Detection - Canny Filter

In [ ]:
fig, axes = plt.subplots(2, 6, figsize=(18, 6))

for i, img in enumerate(open_resized):
    # Original
    axes[0, i*2].imshow(img, cmap='gray')
    axes[0, i*2].set_title(f'Open {i+1} - Original')
    axes[0, i*2].axis('off')
    
    # Canny edges
    edges = cv2.Canny(img, 30, 100)
    axes[0, i*2+1].imshow(edges, cmap='gray')
    axes[0, i*2+1].set_title(f'Canny Edges')
    axes[0, i*2+1].axis('off')

for i, img in enumerate(closed_resized):
    # Original
    axes[1, i*2].imshow(img, cmap='gray')
    axes[1, i*2].set_title(f'Closed {i+1} - Original')
    axes[1, i*2].axis('off')
    
    # Canny edges
    edges = cv2.Canny(img, 30, 100)
    axes[1, i*2+1].imshow(edges, cmap='gray')
    axes[1, i*2+1].set_title(f'Canny Edges')
    axes[1, i*2+1].axis('off')

plt.suptitle('Step 4: Edge Detection - Canny Filter (30, 100)')
plt.tight_layout()
plt.show()

# Calculate edge density
print("🔍 Edge Density Analysis:")
for i, (open_img, closed_img) in enumerate(zip(open_resized, closed_resized)):
    open_edges = cv2.Canny(open_img, 30, 100)
    closed_edges = cv2.Canny(closed_img, 30, 100)
    
    open_density = np.sum(open_edges) / (32 * 32)
    closed_density = np.sum(closed_edges) / (32 * 32)
    
    print(f"Image {i+1} - Open edge density: {open_density:.2f}, Closed edge density: {closed_density:.2f}")

## 📐 Step 5: Gradient Analysis - Sobel Filters

In [ ]:
fig, axes = plt.subplots(3, 6, figsize=(18, 9))

for i, img in enumerate(open_resized):
    # Original
    axes[0, i*2].imshow(img, cmap='gray')
    axes[0, i*2].set_title(f'Open {i+1}')
    axes[0, i*2].axis('off')
    
    # Sobel X
    grad_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
    axes[1, i*2].imshow(np.abs(grad_x), cmap='gray')
    axes[1, i*2].set_title(f'Sobel X')
    axes[1, i*2].axis('off')
    
    # Sobel Y
    grad_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
    axes[2, i*2].imshow(np.abs(grad_y), cmap='gray')
    axes[2, i*2].set_title(f'Sobel Y')
    axes[2, i*2].axis('off')

for i, img in enumerate(closed_resized):
    # Original
    axes[0, i*2+1].imshow(img, cmap='gray')
    axes[0, i*2+1].set_title(f'Closed {i+1}')
    axes[0, i*2+1].axis('off')
    
    # Sobel X
    grad_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
    axes[1, i*2+1].imshow(np.abs(grad_x), cmap='gray')
    axes[1, i*2+1].set_title(f'Sobel X')
    axes[1, i*2+1].axis('off')
    
    # Sobel Y
    grad_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
    axes[2, i*2+1].imshow(np.abs(grad_y), cmap='gray')
    axes[2, i*2+1].set_title(f'Sobel Y')
    axes[2, i*2+1].axis('off')

plt.suptitle('Step 5: Gradient Analysis - Sobel Filters')
plt.tight_layout()
plt.show()

# Calculate gradient statistics
print("📐 Gradient Statistics:")
for i, (open_img, closed_img) in enumerate(zip(open_resized, closed_resized)):
    # Open gradients
    open_grad_x = cv2.Sobel(open_img, cv2.CV_64F, 1, 0, ksize=3)
    open_grad_y = cv2.Sobel(open_img, cv2.CV_64F, 0, 1, ksize=3)
    
    # Closed gradients
    closed_grad_x = cv2.Sobel(closed_img, cv2.CV_64F, 1, 0, ksize=3)
    closed_grad_y = cv2.Sobel(closed_img, cv2.CV_64F, 0, 1, ksize=3)
    
    print(f"Image {i+1}:")
    print(f"  Open - Grad X mean: {np.mean(np.abs(open_grad_x)):.2f}, Grad Y mean: {np.mean(np.abs(open_grad_y)):.2f}")
    print(f"  Closed - Grad X mean: {np.mean(np.abs(closed_grad_x)):.2f}, Grad Y mean: {np.mean(np.abs(closed_grad_y)):.2f}")

## 🔧 Step 6: Morphological Operations

In [ ]:
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))

fig, axes = plt.subplots(4, 6, figsize=(18, 12))

for i, img in enumerate(open_resized):
    # Original
    axes[0, i*2].imshow(img, cmap='gray')
    axes[0, i*2].set_title(f'Open {i+1} - Original')
    axes[0, i*2].axis('off')
    
    # Opening
    opened = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    axes[1, i*2].imshow(opened, cmap='gray')
    axes[1, i*2].set_title(f'Opening')
    axes[1, i*2].axis('off')
    
    # Closing
    closed = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    axes[2, i*2].imshow(closed, cmap='gray')
    axes[2, i*2].set_title(f'Closing')
    axes[2, i*2].axis('off')
    
    # Top-hat (Original - Opening)
    tophat = img - opened
    axes[3, i*2].imshow(tophat, cmap='gray')
    axes[3, i*2].set_title(f'Top-hat')
    axes[3, i*2].axis('off')

for i, img in enumerate(closed_resized):
    # Original
    axes[0, i*2+1].imshow(img, cmap='gray')
    axes[0, i*2+1].set_title(f'Closed {i+1} - Original')
    axes[0, i*2+1].axis('off')
    
    # Opening
    opened = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    axes[1, i*2+1].imshow(opened, cmap='gray')
    axes[1, i*2+1].set_title(f'Opening')
    axes[1, i*2+1].axis('off')
    
    # Closing
    closed_morph = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    axes[2, i*2+1].imshow(closed_morph, cmap='gray')
    axes[2, i*2+1].set_title(f'Closing')
    axes[2, i*2+1].axis('off')
    
    # Top-hat
    tophat = img - opened
    axes[3, i*2+1].imshow(tophat, cmap='gray')
    axes[3, i*2+1].set_title(f'Top-hat')
    axes[3, i*2+1].axis('off')

plt.suptitle('Step 6: Morphological Operations (3x3 Ellipse Kernel)')
plt.tight_layout()
plt.show()

## 📊 Step 7: Histogram Analysis

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

for i, img in enumerate(open_resized):
    hist = cv2.calcHist([img], [0], None, [8], [0, 256])
    axes[0, i].bar(range(8), hist.flatten(), alpha=0.7, color='green')
    axes[0, i].set_title(f'Open Eye {i+1} - 8-bin Histogram')
    axes[0, i].set_xlabel('Intensity Bins')
    axes[0, i].set_ylabel('Pixel Count')

for i, img in enumerate(closed_resized):
    hist = cv2.calcHist([img], [0], None, [8], [0, 256])
    axes[1, i].bar(range(8), hist.flatten(), alpha=0.7, color='red')
    axes[1, i].set_title(f'Closed Eye {i+1} - 8-bin Histogram')
    axes[1, i].set_xlabel('Intensity Bins')
    axes[1, i].set_ylabel('Pixel Count')

plt.suptitle('Step 7: Histogram Analysis (8 bins, 0-256 range)')
plt.tight_layout()
plt.show()

print("📊 Histogram Features Summary:")
for i, (open_img, closed_img) in enumerate(zip(open_resized, closed_resized)):
    open_hist = cv2.calcHist([open_img], [0], None, [8], [0, 256]).flatten()
    closed_hist = cv2.calcHist([closed_img], [0], None, [8], [0, 256]).flatten()
    
    print(f"Image {i+1}:")
    print(f"  Open histogram: {open_hist}")
    print(f"  Closed histogram: {closed_hist}")

## 🔄 Step 8: Symmetry Analysis

In [ ]:
fig, axes = plt.subplots(3, 6, figsize=(18, 9))

for i, img in enumerate(open_resized):
    # Original
    axes[0, i*2].imshow(img, cmap='gray')
    axes[0, i*2].set_title(f'Open {i+1} - Full')
    axes[0, i*2].axis('off')
    
    # Left half
    left_half = img[:, :16]
    axes[1, i*2].imshow(left_half, cmap='gray')
    axes[1, i*2].set_title(f'Left Half')
    axes[1, i*2].axis('off')
    
    # Right half (flipped)
    right_half = cv2.flip(img[:, 16:], 1)
    axes[2, i*2].imshow(right_half, cmap='gray')
    axes[2, i*2].set_title(f'Right Half (flipped)')
    axes[2, i*2].axis('off')

for i, img in enumerate(closed_resized):
    # Original
    axes[0, i*2+1].imshow(img, cmap='gray')
    axes[0, i*2+1].set_title(f'Closed {i+1} - Full')
    axes[0, i*2+1].axis('off')
    
    # Left half
    left_half = img[:, :16]
    axes[1, i*2+1].imshow(left_half, cmap='gray')
    axes[1, i*2+1].set_title(f'Left Half')
    axes[1, i*2+1].axis('off')
    
    # Right half (flipped)
    right_half = cv2.flip(img[:, 16:], 1)
    axes[2, i*2+1].imshow(right_half, cmap='gray')
    axes[2, i*2+1].set_title(f'Right Half (flipped)')
    axes[2, i*2+1].axis('off')

plt.suptitle('Step 8: Symmetry Analysis - Left vs Right Correlation')
plt.tight_layout()
plt.show()

print("🔄 Symmetry Correlation Analysis:")
for i, (open_img, closed_img) in enumerate(zip(open_resized, closed_resized)):
    # Open eye symmetry
    open_left = open_img[:, :16]
    open_right = cv2.flip(open_img[:, 16:], 1)
    open_corr = np.corrcoef(open_left.flatten(), open_right.flatten())[0, 1]
    
    # Closed eye symmetry
    closed_left = closed_img[:, :16]
    closed_right = cv2.flip(closed_img[:, 16:], 1)
    closed_corr = np.corrcoef(closed_left.flatten(), closed_right.flatten())[0, 1]
    
    print(f"Image {i+1} - Open correlation: {open_corr:.3f}, Closed correlation: {closed_corr:.3f}")

## 🎯 Final Feature Vector Comparison

In [ ]:
# Extract complete feature vectors
open_features = [extract_eye_features(img) for img in open_resized]
closed_features = [extract_eye_features(img) for img in closed_resized]

# Feature names
feature_names = [
    'Mean', 'Std', 'Var', 'Min', 'Max', 'Median',  # Statistical (6)
    'Center Mean', 'Center Std', 'Center Contrast',  # Texture (3)
    'Edge Density',  # Edge (1)
    'Grad X Mean', 'Grad Y Mean', 'Grad X Std', 'Grad Y Std',  # Gradient (4)
    'Opening Mean', 'Closing Mean', 'Top-hat Mean', 'Black-hat Mean',  # Morphological (4)
    'Hist Bin 0', 'Hist Bin 1', 'Hist Bin 2', 'Hist Bin 3',  # Histogram (8)
    'Hist Bin 4', 'Hist Bin 5', 'Hist Bin 6', 'Hist Bin 7',
    'Symmetry Correlation'  # Symmetry (1)
]

# Plot feature comparison
fig, axes = plt.subplots(5, 5, figsize=(20, 20))
axes = axes.flatten()

for i in range(25):
    open_vals = [features[i] for features in open_features]
    closed_vals = [features[i] for features in closed_features]
    
    x = np.arange(len(open_vals))
    width = 0.35
    
    axes[i].bar(x - width/2, open_vals, width, label='Open', alpha=0.7, color='green')
    axes[i].bar(x + width/2, closed_vals, width, label='Closed', alpha=0.7, color='red')
    axes[i].set_title(f'{i+1}. {feature_names[i]}')
    axes[i].legend()
    axes[i].set_xticks(x)
    axes[i].set_xticklabels([f'Img {j+1}' for j in range(len(open_vals))])

plt.suptitle('Complete 25-Feature Vector Comparison: Open vs Closed Eyes')
plt.tight_layout()
plt.show()

print("\n🎯 Feature Extraction Summary:")
print(f"✅ Statistical Features: 6 (mean, std, var, min, max, median)")
print(f"✅ Texture Features: 3 (center analysis)")
print(f"✅ Edge Features: 1 (Canny edge density)")
print(f"✅ Gradient Features: 4 (Sobel X/Y mean & std)")
print(f"✅ Morphological Features: 4 (opening, closing, top-hat, black-hat)")
print(f"✅ Histogram Features: 8 (8-bin intensity distribution)")
print(f"✅ Symmetry Features: 1 (left-right correlation)")
print(f"📊 Total Features: 25")

print("\n🔬 Key Observations:")
print("• Open eyes typically have higher intensity variance")
print("• Closed eyes show more uniform pixel distribution")
print("• Edge density differs significantly between open/closed")
print("• Morphological operations reveal structural differences")
print("• Histogram patterns are distinct for each eye state")